# GOALS

- *Top hiring companies (curiosity)*
- Positon Categorization (Analyst, Scientist & Engineer)
- Proportion Data vs Science vs Engineer (Roles)
- Modality Proportion (Trainee, Junior, Mid & Senior)
- Seniority x Category
- Location Percentage
- Keywords Exploration (soft, hard and tools)

# LIBRARIES

In [86]:
import re
import sys
import nltk
import unidecode
import numpy as np
import pandas as pd
import seaborn as sns
from nltk import Text
from textblob import TextBlob
from langdetect import detect
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from nltk.corpus import stopwords
from googletrans import Translator
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
%matplotlib inline
pd.set_option('display.max_colwidth', None)

In [87]:
df = pd.read_csv('DATA_JOBS_JULIO.csv')

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        424 non-null    object 
 1   company      422 non-null    object 
 2   location     423 non-null    object 
 3   modality     406 non-null    object 
 4   description  423 non-null    object 
 5   companies    0 non-null      float64
dtypes: float64(1), object(5)
memory usage: 20.0+ KB


In [89]:
df

title  \
0                                                     Senior Game Data Analyst   
1                                               Data Analyst (Operations Team)   
2                                                                Data Engineer   
3                                                     Analista de Datos Junior   
4                                                                 Data Analyst   
..                                                                         ...   
419                         Arabic Language Data Analyst (Temporary Placement)   
420  Senior Service Architect for Data Management and Landscape Transformation   
421                                       Data & Adtech Account Media Director   
422                                               Danish Language Data Analyst   
423                                   Arabic Language Data Analyst - Barcelona   

                                   company                     location  \
0                            Tilting Point  Barcelona, Catalonia, Spain   
1                                  Holaluz  Barcelona, Catalonia, Spain   
2                                    IOMED  Barcelona, Catalonia, Spain   
3                                      CGI                        Spain   
4                              Olala Homes  Barcelona, Catalonia, Spain   
..                                     ...                          ...   
419  TELUS International AI Data Solutions  Barcelona, Catalonia, Spain   
420                                    SAP  Barcelona, Catalonia, Spain   
421                             Silicon HR  Barcelona, Catalonia, Spain   
422                           TransPerfect  Barcelona, Catalonia, Spain   
423                           TransPerfect  Barcelona, Catalonia, Spain   

    modality  \
0     Hybrid   
1    On-site   
2     Hybrid   
3     Remote   
4     Hybrid   
..       ...   
419  On-site   
420   Hybrid   
421   Hybrid   
422  On-site   
423  On-site   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [90]:
df = df.drop('companies', axis=1)


In [91]:
df = df.dropna(subset=['description'])


# Let's remove duplicates

In [92]:
df = df.drop_duplicates()
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 423 entries, 0 to 423
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        423 non-null    object
 1   company      422 non-null    object
 2   location     423 non-null    object
 3   modality     406 non-null    object
 4   description  423 non-null    object
dtypes: object(5)
memory usage: 19.8+ KB


# Positions' Categorization

In [93]:
df['title'] = df['title'].str.lower()

analysts = len(df[df['title'].str.contains("anal")])
scientists = len(df[df['title'].str.contains("scien")])
engineers = len(df[df['title'].str.contains("engin")])

scientists += len(df[df['title'].str.contains("cient")])
engineers += len(df[df['title'].str.contains("ingen")])

print(f'Analysts: {round(analysts/df.shape[0]*100)}%')
print(f'Scientists: {round(scientists/df.shape[0]*100)}%')
print(f'Engineers: {round(engineers/df.shape[0]*100)}%')

Analysts: 30%
Scientists: 41%
Engineers: 24%


In [94]:
n = 0
overlapped = []
for index, job in df['title'].iteritems():
    if 'scien' in job and 'anal' in job:
        n += 1
        overlapped.append(index)
per = round(n/(analysts+scientists)*100)
print(f'{n} job positions are overlapped, {per}% of all Data Analyst, Data Engineer & Data Scientist positions.')

1 job positions are overlapped, 0% of all Data Analyst, Data Engineer & Data Scientist positions.


In [95]:
df.iloc[overlapped, 0]


84    data scientist (product analytics) esp
Name: title, dtype: object

In [96]:
df.iloc[84, 0] = 'data scientist'


In [97]:
def find_missing_titles(df):
    missing_titles = []
    for title in df['title']:
        if 'anal' not in title and 'scien' not in title and 'engin' not in title:
            missing_titles.append(title)
    return missing_titles

In [98]:
missing_titles = find_missing_titles(df)
for title in missing_titles:
    print(title)

data manager
científico de datos - ia
data operations intern
junior data & business intelligence support specialist
técnico/a de base de datos
database administrator
data consultant
data manager
especialista en bbdd y modelado de datos
data leader (sql - big data)
data quality manager (barcelona)
internship programme data (programa de becas)
senior data consultant
lead data platform
director of data
transformation & data associate
data acquisition strategist (m/f/d)
azure data consultant
data modelling architect
🏠 arquitecto de datos dwh - datamart 🏠
azure data consultant
senior data architect
data specialist
data architect - digital markets
data architect
data manager & statistician, malaria immunology group (short term)
principal data architect & data steward
data manager for the ubioesdm
head of global master data
data manager strategy
database administrator (dba) - remote within spain
data & transformation office leader
data coordinator (fixed term)
product data consultant (m/f/d)


# Let's categorize: Analyst, Scientist and Engineer positions

In [99]:
category = []
dropped = []
for index, position in df['title'].iteritems():
    if "scien" in position:
        category.append('data scientist')
    elif "anal" in position:
        category.append('data analyst')
    elif "engin" in position:
        category.append('data engineer')
    elif "ingeni" in position:
        category.append('data engineer')
    elif "cienti" in position:
        category.append('data scientist')
    else:
        dropped.append(index)

df.drop(dropped, inplace=True)
df.reset_index(inplace=True, drop=True)
df['position'] = category

In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        317 non-null    object
 1   company      316 non-null    object
 2   location     317 non-null    object
 3   modality     302 non-null    object
 4   description  317 non-null    object
 5   position     317 non-null    object
dtypes: object(6)
memory usage: 15.0+ KB


# Data Exploration

In [101]:
df['position'].value_counts()

data analyst      128
data engineer      96
data scientist     93
Name: position, dtype: int64

In [102]:
print(f"Analysts: {round(len(df[df['position'] == 'data analyst'])/df.shape[0]*100)}%")
print(f"Scientists: {round(len(df[df['position'] == 'data scientist'])/df.shape[0]*100)}%")
print(f"Engineer: {round(len(df[df['position'] == 'data engineer'])/df.shape[0]*100)}%")


Analysts: 40%
Scientists: 29%
Engineer: 30%


# Top companies most looking for talent

In [103]:
df['company'].value_counts()


Glovo                            13
eDreams ODIGEO                   10
HP                                9
Zurich Insurance                  9
AstraZeneca                       8
                                 ..
TELUS International               1
Bayer                             1
Logicalis Spain                   1
Moody's Corporation               1
Digital Legends Entertainment     1
Name: company, Length: 161, dtype: int64

# Modality Exploration

In [104]:
df['modality'].value_counts()

Hybrid     168
On-site     66
Remote      42
Unknown     26
Name: modality, dtype: int64

# Location Exploration

In [105]:
df['location'].value_counts()


Barcelona, Catalonia, Spain                    248
Greater Barcelona Metropolitan Area             21
Spain                                           20
Sant Cugat del Vallès, Catalonia, Spain         15
Palau-solità i Plegamans, Catalonia, Spain       4
L'Hospitalet de Llobregat, Catalonia, Spain      3
Esplugues de Llobregat, Catalonia, Spain         2
Cornellà de Llobregat, Catalonia, Spain          1
Mataró, Catalonia, Spain                         1
Mollet del Vallès, Catalonia, Spain              1
Sant Boi de Llobregat, Catalonia, Spain          1
Name: location, dtype: int64

# Cleaning Descriptions

In [106]:
# Replace - jumps with spaces
df['description'] = df['description'].str.replace('\n', ' ')
# Delete the hyphens & ¿
df['description'] = df['description'].map(lambda x: x.replace('-', '').replace('¿', ''))
# Normalise words (no accents, etc.)
df['description'] = df['description'].map(lambda x: unidecode.unidecode(x))
# Adding spaces in a word if it contains a capitalised letter in between
df['description'] = df['description'].map(lambda x: re.sub(r"(?<![A-Z])(?<!^)([A-Z])",r" \1", x))
# Adding spaces before and after numbers
df['description'] = df['description'].map(lambda x: re.sub(r"([0-9]+(\.[0-9]+)?)", r" \1", x))
# Substitute n number of spaces by just one space
df['description'] = df['description'].map(lambda x: ' '.join(x.split()))
# Remove space before dots and after opening parenthesis
df['description'] = df['description'].map(lambda x: x.replace('( ', '(').replace(' .', '.').replace('/ ', '/'))
# Fix problems with the normalisation
df['description'] = df['description'].map(lambda x: x.replace('ano', 'año').replace('anos', 'años'))


In [107]:
df.head(2)

title        company                     location  \
0        senior game data analyst  Tilting Point  Barcelona, Catalonia, Spain   
1  data analyst (operations team)        Holaluz  Barcelona, Catalonia, Spain   

  modality  \
0   Hybrid   
1  On-site   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [108]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import spacy
nltk.download()

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Descargar modelo de SpaCy
spacy.cli.download("es_core_news_sm")

# Cargar modelo de SpaCy
nlp = spacy.load("es_core_news_sm")

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/albertcoca/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/albertcoca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/albertcoca/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 21.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


- Let's convert list to string ('Description' column)

In [109]:
tokenizer = RegexpTokenizer(r'\w+')
df['description'] = df['description'].map(lambda x: tokenizer.tokenize(x.lower()))
# Removing stopwords
stop_words = stopwords.words('english')
#for index, des in df['description'].iteritems():
    #df.iloc[index, 3] =  [w for w in des if not w in stop_words]
# Lemmatization
def lemma(text):
    return [WordNetLemmatizer().lemmatize(w) for w in text]
df['description'] = df['description'].apply(lemma)



In [133]:
df.head(2)

,title,company,location,modality,description,position,seniority
0,senior game data analyst,Tilting Point,"Barcelona, Catalonia, Spain",Hybrid,"[at, tilting, point, we, combine, data, technology, talent, and, groundbreaking, marketing, to, scale, game, to, success, a, the, senior, game, data, analyst, you, will, work, closely, with, game, team, to, help, inform, decisionmaking, regarding, feature, of, the, game, balancing, player, interaction, finding, hidden, opportunity, and, much, more, your, future, at, tilting, point, analyze, large, complex, datasets, to, address, adhoc, and, longterm, operational, and, strategic, business, question, draw, meaningful, conclusion, and, provide, actionable, insight, from, data, to, shape, and, influence, ingame, feature, economy, and, future, product, roadmaps, own, the, taxonomy, for, game, data, ensuring, all, key, performance, metric, are, measurable, and, ...]",data analyst,senior
1,data analyst (operations team),Holaluz,"Barcelona, Catalonia, Spain",On-site,"[description, quienes, somos, somos, holaluz, la, empresa, que, lleva, una, decada, revolucionando, el, sector, energetico, nacimos, para, cambiar, el, mundo, conectando, a, la, persona, con, la, energia, verde, y, tu, y, el, planeta, sois, nuestra, motivacion, para, ser, como, somos, y, hacer, lo, que, hacemos, nuestra, voluntad, de, hacer, la, cosas, de, un, modo, distinto, no, ha, llevado, a, liderar, la, transformacion, del, sector, electrico, y, a, ser, la, primera, electrica, en, obtener, el, certificado, bcorp, te, sumas, unete, a, la, revolucion, que, haras, formaras, parte, del, chapter, de, data, analytics, con, foco, en, el, area, de, operaciones, que, esperamos, ...]",data analyst,unknown


# NLTK - Descriptions and titles

In [111]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')









[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/albertcoca/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/albertcoca/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Descriptions

In [112]:
all_des = []
all_wrds = []
for des in df['description']:
    all_des.append(des)
    for word in des:
        all_wrds.append(word)
all_wrds = [w.replace('visualisation', 'visualization') for w in all_wrds]
all_wrds = [w.replace('modelling', 'modeling') for w in all_wrds]
all_wrds = [w.replace('artificial intelligence', 'ai') for w in all_wrds]
all_wrds = [w.replace('dashboarding', 'dashboard') for w in all_wrds]
all_wrds = [w.replace('kpis', 'kpi') for w in all_wrds]
all_wrds = [w.replace('report', 'reporting') for w in all_wrds]
all_wrds = [w.replace('predict', 'prediction') for w in all_wrds]
all_wrds = [w.replace('creative', 'creativity') for w in all_wrds]
for l in all_wrds:
    if l == 'team' and all_wrds[all_wrds.index(l) + 1] == 'working':
        all_wrds[all_wrds.index(l) + 1] = 'work'
descriptions = Text(all_wrds)

In [113]:
all_des_a = []
all_des_s = []
all_des_e = []
for des in df[df['position'] == 'data analyst']['description']:
    for word in des:
        all_des_a.append(word)
for des in df[df['position'] == 'data scientist']['description']:
    for word in des:
        all_des_s.append(word)
for des in df[df['position'] == 'data engineer']['description']:
    for word in des:
        all_des_e.append(word)



descriptions_a = Text(all_des_a)
descriptions_s = Text(all_des_s)
descriptions_e = Text(all_des_e)

titles

In [114]:
all_titles = []
for title in df['title']:
    for word in title:
        all_titles.append(word)
titles = Text(all_titles)

In [115]:
for index, d in df['description'].iteritems():
    d = [w.replace('visualisation', 'visualization') for w in d]
    d = [w.replace('modelling', 'modeling') for w in d]
    d = [w.replace('artificial intelligence', 'ai') for w in d]
    d = [w.replace('dashboarding', 'dashboard') for w in d]
    d = [w.replace('kpis', 'kpi') for w in d]
    d = [w.replace('report', 'reporting') for w in d]
    d = [w.replace('predict', 'prediction') for w in d]
    d = [w.replace('creative', 'creativity') for w in d]
    df['description'][index] = d

# EXPERIENCE EXPLORATION

In [116]:
#In the title
seniority = 0
juniority = 0

for des in df['title']:
    if 'senior' in des or 'sr' in des or 'head' in des or 'manager' in des or 'director' in des:
        seniority += 1
    elif 'junior' in des or 'jr' in des or 'intern' in des or 'associate' in des or 'trainee' in des:
        juniority += 1

print(seniority)
print(juniority)

86
20


In [117]:
#In the descrption
seniority = 0
juniority = 0

for des in df['description']:
    if 'senior' in des or 'sr' in des:
        seniority += 1
    elif 'junior' in des or 'jr' in des or 'intern' in des or 'associate' in des or 'trainee' in des:
        juniority += 1

print(seniority)
print(juniority)


70
27


# Lets segment it by experience

In [118]:
seniority = []
for i in range(len(df)):
    des = df.loc[i, 'description']
    tit = df.loc[i, 'title']
    if 'senior' in des or 'sr' in des or 'head' in des or 'manager' in des or 'director' in des or 'expert' in des or 'senior' in tit or 'sr' in tit or 'head' in tit or 'manager' in tit or 'director' in tit or 'expert' in tit:
        seniority.append('senior')
    elif 'junior' in des or 'jr' in des or 'intern' in des or 'associate' in des or 'beca' in des or 'trainee' in des or 'junior' in tit or 'jr' in tit or 'intern' in tit or 'associate' in tit or 'trainee' in tit or 'beca' in tit:
        seniority.append('junior')
    else:
        try:
            s = Text(des).concordance_list('year')[0]
            try:
                y = int(s[0][-1])
                if y < 4:
                    seniority.append('junior')
                else:
                    seniority.append('senior')
            except ValueError:
                seniority.append('unknown')
        except IndexError:
            seniority.append('unknown')


In [119]:
df['seniority'] = seniority
df['seniority'].value_counts()

senior     190
unknown     86
junior      41
Name: seniority, dtype: int64

# Keywords (one of the most interesting parts)

TOOLS

In [120]:
tools = ['excel','qlik', 'python', 'r', 'sql', 'git', 'power bi', 'aws', 'azure', 'looker',
        'agile', 'knime', 'matlab', 'tableau']

HARD SKILLS

In [121]:
hard_skills = ['programming', 'machine learning','visualization',
               'modeling','business analyst','research','deep learning', 'analytical skill',
               'optimization', 'automation', 'ab', 'ai', 'dashboard',
               'kpi', 'big data', 'data mining', 'etl', 'reporting', 'segmentation',
               'business intelligence', 'natural language', 'prediction', 'algorithm']

SOFT SKILLS

In [122]:
soft_skills = ['communication', 'client oriented', 'team work', 'presentation', 'management', 'decision making',
              'creativity', 'curiosity', 'leadership', 'problem solving', 'attention detail',
              'motivation', 'proactive', 'empathetic', ]

# EXTRACTION - KW

In [123]:
tfidf = TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w+\\b')
text = [' '.join(doc) for doc in df['description']]
tfidf.fit(text)
dtm = tfidf.transform(text)
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names_out())

In [124]:
key_words = dtm.loc[:, dtm.columns.isin(tools + hard_skills + soft_skills)]


# KEYWORDS' DF

In [128]:
df_key = pd.concat([df, key_words], axis=1)


In [129]:
df_key.drop(['description', 'title'], axis=1, inplace=True)


In [130]:
df_key.groupby('position').agg('mean')


,ab,agile,ai,algorithm,analytical skill,automation,aws,azure,big data,business analyst,...,python,qlik,r,reporting,research,segmentation,sql,tableau,team work,visualization
position,,,,,,,,,,,,,,,,,,,,,
data analyst,0.000585,0.002496,0.005733,0.000542,0.002204,0.004949,0.001567,0.002406,0.002040,0.001204,...,0.005062,0.005286,0.005331,0.009510,0.004404,0.000311,0.009313,0.008016,0.000770,0.008400
data engineer,0.000000,0.006555,0.006417,0.001526,0.000630,0.003518,0.009203,0.012008,0.007388,0.000482,...,0.007604,0.001511,0.002709,0.002546,0.002410,0.000000,0.013258,0.001314,0.001570,0.002397
data scientist,0.001150,0.004598,0.010430,0.010511,0.000319,0.001816,0.006009,0.002889,0.007606,0.000588,...,0.009729,0.000792,0.009414,0.003181,0.009945,0.003765,0.007480,0.002888,0.000264,0.007011


In [131]:
df_key.to_csv('df_keys.csv', index=None, header=True)


In [134]:
test = pd.read_csv('df_keys.csv')
test

,company,location,modality,position,seniority,ab,agile,ai,algorithm,analytical skill,...,python,qlik,r,reporting,research,segmentation,sql,tableau,team work,visualization
0,Tilting Point,"Barcelona, Catalonia, Spain",Hybrid,data analyst,senior,0.0,0.0,0.000000,0.0,0.000000,...,0.012703,0.0,0.019824,0.000000,0.000000,0.000000,0.013639,0.022108,0.0,0.020491
1,Holaluz,"Barcelona, Catalonia, Spain",On-site,data analyst,unknown,0.0,0.0,0.000000,0.0,0.000000,...,0.002768,0.0,0.000000,0.000000,0.000000,0.000000,0.002971,0.000000,0.0,0.000000
2,IOMED,"Barcelona, Catalonia, Spain",Hybrid,data engineer,junior,0.0,0.0,0.028947,0.0,0.000000,...,0.014619,0.0,0.000000,0.000000,0.027534,0.000000,0.023544,0.000000,0.0,0.000000
3,CGI,Spain,Remote,data analyst,senior,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,Olala Homes,"Barcelona, Catalonia, Spain",Hybrid,data analyst,junior,0.0,0.0,0.000000,0.0,0.022596,...,0.000000,0.0,0.000000,0.040021,0.000000,0.022961,0.008405,0.000000,0.0,0.037882
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312,TransPerfect,"Barcelona, Catalonia, Spain",Hybrid,data analyst,unknown,0.0,0.0,0.027027,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
313,Digital Legends Entertainment,"Barcelona, Catalonia, Spain",Hybrid,data engineer,senior,0.0,0.0,0.000000,0.0,0.000000,...,0.011205,0.0,0.034971,0.000000,0.021104,0.000000,0.000000,0.000000,0.0,0.000000
314,TELUS International AI Data Solutions,"Barcelona, Catalonia, Spain",On-site,data analyst,unknown,0.0,0.0,0.095046,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
315,TransPerfect,"Barcelona, Catalonia, Spain",On-site,data analyst,unknown,0.0,0.0,0.028599,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
